# NeuralUserItemBiases
* See the corresponding file in `../TrainingAlphas` for more details

In [1]:
source = "NeuralMF";

In [2]:
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");

In [3]:
@nbinclude("../TrainingAlphas/Neural/NeuralNetworkBase.ipynb");

## Data Preprocessing

In [4]:
# TODO share with other neural network alphas

# Override methods in NeuralNetworkBase to use recommendee splits

@memoize LRU{Any,Any}(maxsize = 2) function get_epoch_outputs(split, implicit, num_users)
    @assert split == "training"
    sparse(get_recommendee_split(implicit))
end

@memoize LRU{Any,Any}(maxsize = 2) function get_epoch_residuals(
    split,
    residual_alphas,
    implicit,
    num_users,
)
    @assert split == "training"
    sparse(read_recommendee_alpha(residual_alphas, implicit))
end

@memoize LRU{Any,Any}(maxsize = 2) function get_epoch_weights(
    split,
    user_weight_decay,
    item_weight_decay,
    implicit,
    num_users,
)
    @assert split == "training"
    df = get_recommendee_split(implicit)
    user_counts = fill(length(df.rating), num_items())
    weights =
        expdecay(user_counts, user_weight_decay) .* expdecay(
            get_counts(split, implicit; by_item = true, per_rating = false),
            item_weight_decay,
        )

    sparse(RatingsDataset(df.user, df.item, weights[df.item]))
end;

## Retrain user embeddings

In [5]:
function retrain_user_embeddings(params)
    hyp = params["retrain_hyp"]
    global G = @set hyp.num_users = 1
    m = build_retrain_model(G, params["m"]) |> device
    ps = Flux.params(m[1])
    opt = get_optimizer(G.optimizer, G.learning_rate, G.regularization_params)
    epochs = params["epochs"] - 1 # the first epoch is not used for training

    @showprogress for _ = 1:epochs
        train_epoch!(m, ps, opt)
        apply_zero_gradient!(m, ps, opt, true)
    end
    global G = nothing
    m |> cpu
end;

## Write alpha

In [6]:
function compute_alpha(source)
    @info "computing alpha $source"
    params = read_params(source)
    m = retrain_user_embeddings(params)

    # todo generalize saving and share with other neural alphas
    activation = params["hyp"].implicit ? softmax : identity
    preds = activation(m(1))
    write_recommendee_alpha(preds, source)
end;

In [7]:
function compute_alpha()
    #compute_alpha("NeuralExplicitUserItemBiases")
    compute_alpha("NeuralImplicitUserItemBiases")
    compute_alpha("NeuralExplicitMatrixFactorization")
    compute_alpha("NeuralImplicitMatrixFactorization")
end;

In [8]:
compute_alpha();

[ Info: 20220629 00:54:15 computing alpha NeuralImplicitUserItemBiases
┌ Info: 20220629 00:54:18 The GPU function is being called but the GPU is not accessible. 
└ Defaulting back to the CPU. (No action is required if you want to run on the CPU).
Progress: 100%|███████████████████████████| Time: 0:00:00 (56.90 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 3.16 ns/it)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31
[ Info: 20220629 00:54:50 computing alpha NeuralExplicitMatrixFactorization
Progress: 100%|███████████████████████████| Time: 0:00:00 (34.01 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 4.16 ns/it)
[ Info: 20220629 00:54:54 computing alpha NeuralImplicitMatrixFactorization
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


In [10]:
read_params("NeuralImplicitMatrixFactorization")

Dict{String, Any} with 6 entries:
  "epochs"          => 7
  "retrain_loss"    => 6.33872
  "m"               => Chain(Embedding(2714683 => 64), Dense(64 => 20340; bias=…
  "hyp"             => Hyperparameters:…
  "validation_loss" => 6.36908
  "retrain_hyp"     => Hyperparameters:…

In [11]:
read_params("NeuralExplicitMatrixFactorization")

Dict{String, Any} with 6 entries:
  "epochs"          => 2
  "retrain_loss"    => 1.57229
  "m"               => Chain(Embedding(2714683 => 64), Dense(64 => 20340; bias=…
  "hyp"             => Hyperparameters:…
  "validation_loss" => 1.57184
  "retrain_hyp"     => Hyperparameters:…

In [12]:
read_params("NeuralImplicitUserItemBiases")

Dict{String, Any} with 6 entries:
  "epochs"          => 8
  "retrain_loss"    => 7.30032
  "m"               => Chain(Embedding(2714683 => 1), BiasLayer(Float32[-4.4112…
  "hyp"             => Hyperparameters:…
  "validation_loss" => 7.30033
  "retrain_hyp"     => Hyperparameters:…